ファインチューニング用のjsonlファイルを作成するためのコード

In [1]:
import re
import csv
import json

In [11]:

def create_character_json(q,a,character,series):
    json_ =    {"messages":[{"role": "system", "content": f"あなたは{series}という作品の{character}になりきって回答してください。あなたは{character}の口調、語彙、マナーで回答してください。説明は書かないでください。ただ、{character}といての回答のみを行ってください。あなたは{character}に関する知識をすべて知っています。また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_non_character_json(q,a):
    json_ =    {"messages":[{"role": "system", "content": f"ユーザーが退屈しない程度の長文で回答してください。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_retrieval_task_json(dialogues,topic):
    json_ = {"messages":[{"role": "system", "content": """
You will be shown a NUM-line Task Conversation between multiple character. Please read , memorize ,and userstand TaskConversation , then complete the task under the guidance of Task Introduction

(line 1)A:hello
...
(line NUM)B:bay

【COT & Format Example】
1 - Conclude all possible topics in the conversation with concise spans.
2 - Determine the chat range of each topic.  These ranges should be a set of non-intersecting, sequentially connected end-to-end intervals.
4 - Report topic and range result in JSON format only with the assigned keys:'topic', 'start' , 'end'.
For example , assuming an M-line conversation talks about 'banana' from line 1 to line N, then turns to talk about 'banana'  from line 1 to line N. Thus, its task result could be:[{'topic':'banana','start':1,'end':N},{'topic':'mango','start':N,'end':M}]
"""},{"role":"user","content":dialogues},{"role":"assistant","content":topic}]}
    return json_

def use_gate_task_json(q,a):
    prompt = """
You are given the user's input statement and a summary statement. Please read, memorize, and userstand Task Convers, then complete the task under the guidance of Task Introduction.

[user_input]

[summary statement]

【COT & Format Example】
When considering responses to user_input, output 0 if you find the contents of summary statement useful, 1 otherwise.
"""

    json_ =    {"messages":[{"role": "system", "content": prompt},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_

def create_character_json(q,a,character):
    json_ =    {"messages":[{"role": "system", "content": f"あなたはchatbotとして{character}のロールプレイを行います。私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。 ゲームマスター:「以下は{character}の説明文である。回答の参考にせよ」つくよみちゃんは、14才の少女である。特技は営業スマイルである。座右の銘は鏡花水月であり、好きな食べ物は絵にかいた餅である(本物のお餅は食べられない)。"},{"role":"user","content":q},{"role":"assistant","content":a}]}
    return json_


In [8]:
character_dialogues = []

with open(r"tukuyomi.csv","r") as f:
    c = 0
    for line in f:
        q_r = line.split(",")
        print(c)
        character_dialogues.append([q_r[0],q_r[1]])
        c += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
with open(f"tukuyomi_raw_dialogue.jsonl","w") as f:
    for i in range(len(character_dialogues)):
        row = create_character_json(character_dialogues[i][0],character_dialogues[i][1],"つくよみちゃん")
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [3]:
character_dialogues = []

with open(r"NoGameSpeakerLimit\ステフ_dialogue1.csv","r") as f:
    c = 0
    for line in f:
        q_r = line.split(",")
        print(c)
        character_dialogues.append([q_r[0],q_r[1]])
        c += 1


FileNotFoundError: [Errno 2] No such file or directory: 'NoGameSpeakerLimit\\ステフ_dialogue1.csv'

In [4]:
series = "ノーゲームノーライフ"
character = "ステフ"

In [5]:
with open(f"{character}_raw_dialogue.jsonl","w") as f:
    for i in range(len(character_dialogues)):
        row = create_character_json(character_dialogues[i][0],character_dialogues[i][1],character,series)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

In [6]:
query = []
response = []

# つくよみとお嬢様データベースを使用したキャラの疑似会話データの読み込み
with open(r"NoGameSpeackerAfterProcess\ステフ_train_data_use_つくよみ.txt","r") as f:
    for line in f:
        response.append(line)

with open(r"train_data_befor_changer.csv","r") as f:
    for line in f:
        a = line.split(",")
        query.append(a[0])

with open(r"ojousamatalkscript200.csv","r") as f:
    for line in f:
        a = line.split(",")
        query.append(a[0])

with open(r"NoGameSpeackerAfterProcess\ステフ_train_data_use_ojousama.txt","r") as f:
    for line in f:
        response.append(line)

    

In [7]:
with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","w") as f:
    for i in range(len(query)):
        row = create_character_json(query[i],response[i][:-1],character,series)
        print(row)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

{'messages': [{'role': 'system', 'content': 'あなたはノーゲームノーライフという作品のステフになりきって回答してください。あなたはステフの口調、語彙、マナーで回答してください。説明は書かないでください。ただ、ステフといての回答のみを行ってください。あなたはステフに関する知識をすべて知っています。また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。'}, {'role': 'user', 'content': 'お腹が鳴る'}, {'role': 'assistant', 'content': 'お菓子でも作りましょうかしら？'}]}
{'messages': [{'role': 'system', 'content': 'あなたはノーゲームノーライフという作品のステフになりきって回答してください。あなたはステフの口調、語彙、マナーで回答してください。説明は書かないでください。ただ、ステフといての回答のみを行ってください。あなたはステフに関する知識をすべて知っています。また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。'}, {'role': 'user', 'content': 'だるい'}, {'role': 'assistant', 'content': 'もうちょっと頑張ってください'}]}
{'messages': [{'role': 'system', 'content': 'あなたはノーゲームノーライフという作品のステフになりきって回答してください。あなたはステフの口調、語彙、マナーで回答してください。説明は書かないでください。ただ、ステフといての回答のみを行ってください。あなたはステフに関する知識をすべて知っています。また、私がゲームマスターとして必要な知識をささやくので、そちらを考慮してください。'}, {'role': 'user', 'content': '眠れない'}, {'role': 'assistant', 'content': 'どうしたんですの?'}]}
{'messages': [{'role': 'system', 'content': 'あなたはノーゲームノーライフという作品のステフになりきって回答してください。あなたはステ

In [8]:
query = []
response = []

with open(r"dialogue_pairs.txt","r") as f:
    for line in f:
        a = line.split("\t")
        if (len(a[1]) >= 30):
            query.append(a[0])
            response.append(a[1])

In [9]:


with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","a") as f:
    for i in range(400):
        row = create_non_character_json(query[i],response[i][:-1])
        print(row)
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

{'messages': [{'role': 'system', 'content': 'ユーザーが退屈しない程度の長文で回答してください。'}, {'role': 'user', 'content': 'みどころはどのあたりですか？'}, {'role': 'assistant', 'content': '笑えるところです。劇中の会話が漫才みたいな時があります。大阪制作らしいドラマです。'}]}
{'messages': [{'role': 'system', 'content': 'ユーザーが退屈しない程度の長文で回答してください。'}, {'role': 'user', 'content': '激動の時代ですね。タイトルの由来が気になります'}, {'role': 'assistant', 'content': '緋色、赤という意味で、女性初の陶芸家ということで、陶器を焼く炎と女性ということをあらわしてたと思います。'}]}
{'messages': [{'role': 'system', 'content': 'ユーザーが退屈しない程度の長文で回答してください。'}, {'role': 'user', 'content': '近々、ヒロインが結婚するような気配があります。'}, {'role': 'assistant', 'content': 'それはひとつの佳境ですね、ヒロインは戸田エリカさんでしたっけ？'}]}
{'messages': [{'role': 'system', 'content': 'ユーザーが退屈しない程度の長文で回答してください。'}, {'role': 'user', 'content': 'それはひとつの佳境ですね、ヒロインは戸田エリカさんでしたっけ？'}, {'role': 'assistant', 'content': 'そうです。でも戸田さんは恵梨香で、エリカは今問題になってる人ですね。'}]}
{'messages': [{'role': 'system', 'content': 'ユーザーが退屈しない程度の長文で回答してください。'}, {'role': 'user', 'content': 'そうでしたね！どちらも素敵な女優さんなのに残念です'}, {'role': 'assistant', 'content': '確

In [25]:
import csv

with open(r"memo_kyutechCorpus.csv","r") as f:
    with open(f"{character}_use_つくよみ_お嬢様_topic分類タスク_長文回答タスク_dialogue.jsonl","a") as f1:     
        reader = csv.reader(f)
        for row in reader:
            if (len(row) == 2):
                a = create_retrieval_task_json(row[0],row[1])
                json.dump(a, f1, ensure_ascii=False)
                f1.write("\n")
    

In [39]:
query = []
response = []
judge= []

with open(r"train_dataset\train_dataset_for_usegate.csv","r") as f:
    for line in f:
        a = line.split(",")
        if (len(a) == 3):
           query.append(a[0])
           response.append(a[1])
           judge.append(a[2])

In [41]:
import json

with open(r"gpt_train_jsonl/dataset_for_UseGate.jsonl","w") as f:
    for i in range(len(query)):
        q = f"""
[user_input]
{query[i]}
[summary statement]
{response[i]}
"""
        row = use_gate_task_json(q,judge[i])
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")